
This notebook is used to test two different deep neural networks on one datasets



In [228]:
#install the necessary modules
!pip install torch
!pip install torchvision
!pip install matplotlib
!pip install scikit-learn

In [229]:
import torch
from torch import nn
from torch import optim
from torchvision import datasets, transforms
from torch.utils.data import random_split, DataLoader

In [230]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import json
import time


In [231]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))  ,
    # transforms.RandomRotation(10),
])


In [232]:
"""
The MNIST dataset is divided into two main folders: one for training and one for testing. To further improve the model's evaluation, 
I will create a validation set by extracting 5% of the training data. 
This validation set will allow me to assess the model's accuracy during training without using the test set. 
By validating on data the model has never encountered, I can better determine if the model is overfitting, ensuring a more generalizable performance.
        Validation set size: 3,000
        Test set size: 10,000
        Training set size: 57,000
"""
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# Function to split the dataset
def dataset_split(train_dataset, val_ratio=0.05):
    # Calculate validation size based on the ratio
    val_size = int(len(train_dataset) * val_ratio)
    train_size = len(train_dataset) - val_size
    
    train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])
    
    val_loader = DataLoader(dataset=val_dataset, batch_size=64, shuffle=False)
    train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
    
    return val_loader, train_loader

val_loader, train_loader = dataset_split(train_dataset, val_ratio=0.05)

test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)


10000
3000
57000


In [233]:
class NetworkA(nn.Module):
  def __init__(self):
    super(NetworkA, self).__init__()
    self.flatten = nn.Flatten()
    self.fc1 = nn.Linear(784,64)
    self.fc2 = nn.Linear(64,128)
    self.fc3 = nn.Linear(128,64)
    self.fc4 = nn.Linear(64,10)
    self.relu = nn.ReLU()
    self.softmax = nn.Softmax(dim=1)
  def forward(self,x):
    x = self.flatten(x)
    x = self.relu(self.fc1(x))
    x = self.relu(self.fc2(x))
    x = self.relu(self.fc3(x))
    x = self.fc4(x)
    return x

In [234]:
class NetworkB(nn.Module):
  def __init__(self):
    super(NetworkB, self).__init__()
    self.flatten = nn.Flatten()
    self.fc1 = nn.Linear(784,256)
    self.fc2 = nn.Linear(256,512)
    self.fc3 = nn.Linear(512,128)
    self.fc4 = nn.Linear(128,10)
    self.relu = nn.ReLU()
    self.softmax = nn.Softmax(dim=1)
  def forward(self,x):
    x = self.flatten(x)
    x = self.relu(self.fc1(x))
    x = self.relu(self.fc2(x))
    x = self.relu(self.fc3(x))
    x = self.fc4(x)
    return x


[link text](https://)

In [236]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

models = [None, None]
models[0] = NetworkA().to(device)
models[1] = NetworkB().to(device)

In [237]:
def train_and_evaluate(model, train_loader, test_loader,num_epochs): 
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    loss_fn = nn.CrossEntropyLoss()
    # To store training data for plotting
    training_data = {
        'train_loss': [],
        'test_loss': [],
        'train_error': [],
        'test_error': [],
        'accuracy': [],
        'precision': [],
        'recall': [],
        'f1_score': []
    }

    start_time = time.time()

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        total_train_loss = 0
        correct_train = 0
        total_train = 0
        
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()

            total_train_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()

        avg_train_loss = total_train_loss / len(train_loader.dataset)
        train_error = 1 - (correct_train / total_train)

        # Testing phase
        model.eval()
        total_test_loss = 0
        correct_test = 0
        total_test = 0
        all_labels = []
        all_preds = []
        
        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)

                outputs = model(images)
                loss = loss_fn(outputs, labels)
                total_test_loss += loss.item() * images.size(0)

                _, predicted = torch.max(outputs.data, 1)
                total_test += labels.size(0)
                correct_test += (predicted == labels).sum().item()

                all_labels.extend(labels.cpu().numpy())
                all_preds.extend(predicted.cpu().numpy())

        avg_test_loss = total_test_loss / len(test_loader.dataset)
        test_error = 1 - (correct_test / total_test)

        accuracy = accuracy_score(all_labels, all_preds)
        precision = precision_score(all_labels, all_preds, average='weighted')
        recall = recall_score(all_labels, all_preds, average='weighted')
        f1 = f1_score(all_labels, all_preds, average='weighted')

        # Store metrics
        training_data['train_loss'].append(avg_train_loss)
        training_data['test_loss'].append(avg_test_loss)
        training_data['train_error'].append(train_error)
        training_data['test_error'].append(test_error)
        training_data['accuracy'].append(accuracy)
        training_data['precision'].append(precision)
        training_data['recall'].append(recall)
        training_data['f1_score'].append(f1)

        print(f'Epoch [{epoch+1}/{num_epochs}], '
              f'Train Loss: {avg_train_loss:.4f}, Test Loss: {avg_test_loss:.4f}, '
              f'Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, '
              f'Recall: {recall:.4f}, F1 Score: {f1:.4f}')

    end_time = time.time()
    training_time = end_time - start_time
    training_data['training_time'] = training_time

    return training_data


In [242]:
def save_model_and_results(model, model_name, training_data):
    torch.save(model.state_dict(), f'{model_name}_model.pth')
    # Convert training data to a JSON-serializable format
    serializable_data = {}
    for key, value in training_data.items():
        if isinstance(value, torch.Tensor):
            serializable_data[key] = value.tolist()  # Convert Tensors to lists
        elif isinstance(value, list) and isinstance(value[0], torch.Tensor):
            serializable_data[key] = [v.tolist() for v in value]  # Convert lists of Tensors
        else:
            serializable_data[key] = value

    # Save the training data as JSON
    with open(f'{model_name}_training_data.json', 'w') as json_file:
        json.dump(serializable_data, json_file)


In [243]:
for idx, model in enumerate(models):
    print(f'Training model {idx + 1}: {model}')
    training_data = train_and_evaluate(model, train_loader, test_loader, num_epochs=500)

    save_model_and_results(model, f'model_{idx + 1}', training_data)


Training model 1: NetworkA(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=784, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=10, bias=True)
  (relu): ReLU()
  (softmax): Softmax(dim=1)
)
Training model 2: NetworkB(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=784, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=128, bias=True)
  (fc4): Linear(in_features=128, out_features=10, bias=True)
  (relu): ReLU()
  (softmax): Softmax(dim=1)
)


In [241]:
import torch

def evaluate_model(model, val_loader):
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

# Loop through both models to evaluate them on the whole validation set
for i, model in enumerate(models):
    accuracy = evaluate_model(model, val_loader)
    print(f'Accuracy for Network{i+1} on the whole validation set: {accuracy:.2%}')


Accuracy for Network1 on the whole validation set: 97.67%
Accuracy for Network2 on the whole validation set: 98.13%
